In [1]:
import pandas as pd
import pickle
import numpy as np

In [2]:
# load model
with open('gb_model.pkl', 'rb') as file:
    model = pickle.load(file)

# load data
df = pd.read_csv('atp_tennis_engineered.csv', index_col=0, parse_dates=True)

C:\Users\adamv\AppData\Local\Temp\ipykernel_16532\744581355.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv('atp_tennis_engineered.csv', index_col=0, parse_dates=True)


In [ ]:
player_A = 'Alcaraz C.'
player_B = 'Fearnley J.'

surface = 'Grass'

# find all matches containing player A and player B as either Winner or Loser
matches_A = df[(df['Winner'] == player_A) | (df['Loser'] == player_A)]
matches_B = df[(df['Winner'] == player_B) | (df['Loser'] == player_B)]

# find rank of players in the last match they played
last_match_A = matches_A.sort_values('Date').iloc[-1]
last_match_B = matches_B.sort_values('Date').iloc[-1]
rank_A = last_match_A['WRank'] if last_match_A['Winner'] == player_A else last_match_A['LRank']
rank_B = last_match_B['WRank'] if last_match_B['Winner'] == player_B else last_match_B['LRank']

# find win proportion of each player in 90 days up to last match in dataset
last_date = pd.to_datetime(df['Date'].max())
form_start_date = (last_date - pd.DateOffset(days=90)).date()
form_matches_A = matches_A[pd.to_datetime(matches_A['Date']).dt.date >= form_start_date]
form_matches_B = matches_B[pd.to_datetime(matches_B['Date']).dt.date >= form_start_date]

form_A = (form_matches_A['Winner'] == player_A).sum() / len(form_matches_A) if len(form_matches_A) > 0 else 0
form_B = (form_matches_B['Winner'] == player_B).sum() / len(form_matches_B) if len(form_matches_B) > 0 else 0

# find win proportion of each player on the given surface
surface_matches_A = matches_A[matches_A['Surface'] == surface]
surface_matches_B = matches_B[matches_B['Surface'] == surface]
surfacewinproportion_A = (surface_matches_A['Winner'] == player_A).sum() / len(surface_matches_A) if len(surface_matches_A) > 0 else 0
surfacewinproportion_B = (surface_matches_B['Winner'] == player_B).sum() / len(surface_matches_B) if len(surface_matches_B) > 0 else 0

# find head-to-head win proportion of each player
h2h_matches = df[((df['Winner'] == player_A) & (df['Loser'] == player_B)) | ((df['Winner'] == player_B) & (df['Loser'] == player_A))]
h2h_A = (h2h_matches['Winner'] == player_A).sum() / len(h2h_matches) if len(h2h_matches) > 0 else 0.5
h2h_B = 1 - h2h_A

features_df = pd.DataFrame(
    data=[[rank_A, rank_B, surfacewinproportion_A, surfacewinproportion_B, h2h_A, h2h_B, form_A, form_B]],
    columns=model.metadata['features']
)

print(features_df.to_string(index=False))
prediction_array = model.predict(features_df)
predicted_winner = player_A if prediction_array[0] == 1 else player_B
print('=======================================')
print(f"Predicted winner: {predicted_winner}")

prediction_array = model.predict(features_df)
predicted_winner = player_A if prediction_array[0] == 1 else player_B
print('=======================================')
print(f"Predicted winner: {predicted_winner}")

 Rank A  Rank B  SurfaceWinProportion A  SurfaceWinProportion B  H2HWinProportion A  H2HWinProportion B   Form A  Form B
    2.0    68.0                0.888889                0.333333                 0.5                 0.5 0.833333     0.5
Predicted winner: Alcaraz C.


In [10]:
df.loc[df['Winner'].str.contains('Djokovic')]['Winner'].unique()

array(['Djokovic N.', 'Djokovic N. '], dtype=object)